# Step 1:

## Load the libraries

In [1]:
import pandas as pd  
import numpy as np
from pandas import read_excel
import pandas as pd
import scipy
from scipy import stats
import os
from glob import glob

# Step 2: Computing the statistics for DMSO wells
## a) Read the dataframe with mean aggregated features

In [10]:
feature_folder = '/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W/'
annot_folder = '/4tbint/Plate_annotations_newMoAs/'

for filename in sorted(os.listdir(feature_folder)):
    plate_path = os.path.join(feature_folder, filename)
    #print(plate_path)
    file = filename.split(".")[0]
    print(file)
    feature_data = pd.read_excel(plate_path)
    prefix = file.split(".")[0]
    #print(prefix)

    for annot_file in sorted(os.listdir(annot_folder)):
        ann_file = annot_file.split(".")
        ann_file = annot_file.split("_annot")[0]
        #print(ann_file)
        annot_path = os.path.join(annot_folder, annot_file)
        #print(annot_path)
        if file == ann_file:
            print(annot_file)

            annot_data = pd.read_excel(annot_path)
                
            feature_data = feature_data.join(annot_data, lsuffix='_left', rsuffix='_right')
            #Put values as "dmso" as we are only interested in rows with Label as "dmso"
            values =['DMSO'] 
            #Search for rows where column Label is in Values
            data_dmso = feature_data[feature_data.Label.isin(values) == True]
            #Select all rows and all columns from those in the "values" variable 
            data_dmso = data_dmso.loc[:,:]

            data_dmso = data_dmso.reset_index()

            #For CP dataset
            #data_dmso = data_dmso.drop(['index','FileName_Cancer','FileName_Cellpose','FileName_Dapi','FileName_Fibroblast','ImageNumber','Well_number','Compound', 'MoA', 'Concentration', 'Label', 'Category'], axis=1) 
            # For TL dataset ---> 
            data_dmso = data_dmso.drop(['Unnamed: 0','index', 'Image', 'Well_number','Compound', 'MoA', 'Concentration', 'Label', 'Category'], axis=1)
       

            #create new dataframe with the same column names as "data_dmso" to have feature cols
            stats_dmso = pd.DataFrame(columns=data_dmso.columns)

            #Compute median and median absolute deviation along the feature columns
            data_dmso_median = data_dmso.median()
            data_dmso_mad = data_dmso[data_dmso.columns[0:(len(data_dmso.columns))]].apply(stats.median_abs_deviation)

            #Append the list of median and median absolute deviation to the dataframe "stats_dmso"
            stats_dmso = stats_dmso.append(pd.Series(data_dmso_median.tolist(), index=stats_dmso.columns), ignore_index=True)
            stats_dmso = stats_dmso.append(pd.Series(data_dmso_mad.tolist(), index=stats_dmso.columns), ignore_index=True)

            #Add the column "Statistics" in the stats_dmso dataframe
            stats_dmso.insert(0, "Statistics", ['Median', 'Median absolute deviation'], True)


            #Take only feature columns from the median aggregated well dataframe (basically originally dataframe that we read in the beginning)

            #For CP Dataset
            #data_features = feature_data.iloc[0:384,5:713] 

            #For TL dataset
            data_features = feature_data.iloc[0:384,2:514] 

            #Subtract the median row from 'stats_dmso' to the original data
            features_sub_median = data_features.subtract(stats_dmso.iloc[0,1:], axis = 1) 
            #for TL: stats_dmso.iloc[0,1:]
            #For CP: dataset stats_dmso.iloc[0,1:709]

            #Divide the median subtracted dataframe from dmso MAD row in the 'stats_dmso' dataframe
            features_divide_mad = features_sub_median.div(stats_dmso.iloc[1,1:], axis = 1) 
            #for TL: stats_dmso.iloc[1,1:]
            #For CP: dataset stats_dmso.iloc[1,1:709]

            #Add the Label column annotation to the dataframe print it and save it as excel file
            normalized_data = features_divide_mad.join(feature_data['Label'])

            normalized_data = normalized_data.iloc[:,0:len(normalized_data.columns)-1].join(annot_data, lsuffix='_left', rsuffix='_right')

            normalized_data_filtered = normalized_data[normalized_data.columns.drop(list(normalized_data.filter(regex='_X')))]
            normalized_data_filtered = normalized_data_filtered[normalized_data_filtered.columns.drop(list(normalized_data_filtered.filter(regex='_Y')))]
            #normalized_data_filtered = normalized_data_filtered.drop(['index'])

    #Save the final dataframe with median aggregated and normalized features at well-level for analysis
    normalized_data_filtered.to_excel("/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized/{}.xlsx".format(prefix))
    

Plate_1
Plate_1_annot.xlsx
Plate_2
Plate_2_annot.xlsx
Plate_3
Plate_3_annot.xlsx
Plate_4
Plate_4_annot.xlsx
Plate_5
Plate_5_annot.xlsx
Plate_6
Plate_6_annot.xlsx
Plate_7
Plate_7_annot.xlsx
Plate_8
Plate_8_annot.xlsx


In [21]:
cd /home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized/


/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized


In [22]:
rm -r .ipynb_checkpoints

# Merge all the plates of all the combinations in one single dataframe 

In [27]:
appended_data = []

norm_features = "/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized/"

for feature_file in sorted(os.listdir(norm_features)):
    print(feature_file)
    norm_feature_data = pd.read_excel(os.path.join(norm_features, feature_file))
    
    #prefix_comb = feature_file.split("_")[1]
    prefix_comb = feature_file.split(".")[0]
    print(prefix_comb)

    if "A81" in prefix_comb:
        combo = "O3B"
        
    elif "A82" in prefix_comb:
        combo = 'KB'
    
    elif "Set25" in prefix_comb:
        combo = 'KW'
        
    elif "Set89" in prefix_comb:
        combo = 'O8W'
        
    elif "863" in prefix_comb:
        combo = 'MHB'
        
    combination = combo
    #print(combination)

    norm_feature_data.insert(0,'Combination', combination)
    
    #prefix_plate = feature_file.split("_")[1]
    #prefix_plate = prefix_comb.split("_")[1]
    prefix_plate = list(prefix_comb)
    #print(prefix_plate)

    norm_feature_data.insert(0,'Plate number', prefix_plate[1] )

    appended_data.append(norm_feature_data)
    
appended_data = pd.concat(appended_data)
# write DataFrame to an excel sheet 
appended_data.to_excel('/home/gauss/Desktop/Trained Features_FilteredBBox_pooledwellsd2d3d4/O8W_normalized/alldata.xlsx')


N1Set89.xlsx
N1Set89
N2Set89.xlsx
N2Set89
N3Set89.xlsx
N3Set89
N4Set89.xlsx
N4Set89
N5Set89.xlsx
N5Set89
N6Set89.xlsx
N6Set89
N7Set89.xlsx
N7Set89
N8Set89.xlsx
N8Set89


In [ ]:
# Read the all data file and replace all null, -inf values by 0
file_name = "/home/gauss/Desktop/Trained Features_FilteredBBox_merged_plates/O3B_normalized/alldata.xlsx"
data = pd.read_excel(file_name)

data

In [ ]:
data = data.drop(['Cancer_Count_left','Cancer_Count_right'], axis=1)


In [ ]:
data

In [ ]:
df2=data.fillna(0)
df2.replace([np.inf, -np.inf], 0, inplace=True)
df2

In [ ]:
data.to_excel('/home/gauss/Desktop/Pretrained Features/MHB_Features_Normalized/alldata.xlsx')
